In [ ]:
import pandas as pd
import numpy  as np
from math import sin, cos, pi, sqrt
import random
import time
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()

_n_paths_         = 5
_n_circles_       = 100
_radius_min_      = 20
_radius_max_      = 30
_min_circle_sep_  = 30
_half_sep_        = _min_circle_sep_/2.0   # Needs to be more than the _radius_inc_test_
_radius_inc_test_ = 4
_radius_start_    = _radius_inc_test_ + 1  # Needs to be more than the _radius_inc_test_ ... less than the _min_circle_sep_
_escape_px_       = 10                     # less than the _min_circle_sep_

def circleOverlaps(cx, cy, r):
    for _geom_ in _circle_geoms_:
        dx, dy = _geom_[0] - cx, _geom_[1] - cy
        d      = sqrt(dx*dx+dy*dy)
        if d < (r + _geom_[2] + _min_circle_sep_): # at least 10 pixels apart...
            return True
    return False

def findOpening():
    _max_attempts_ = 100
    attempts  = 0
    cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), \
                random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
    while circleOverlaps(cx,cy,r) and attempts < _max_attempts_:
        cx, cy, r = random.randint(_radius_max_+_min_circle_sep_, 600-_radius_max_-_min_circle_sep_), \
                    random.randint(_radius_max_+_min_circle_sep_, 400-_radius_max_-_min_circle_sep_), random.randint(_radius_min_,_radius_max_)
        attempts += 1
    if attempts == _max_attempts_:
        return None
    return cx, cy, r

# Create the circles
_circle_geoms_ = []
for i in range(_n_circles_):
    to_unpack = findOpening()
    if to_unpack is not None:
        cx, cy, r = to_unpack
        _circle_geoms_.append((cx,cy,r))

# Randomize the entry point
c0         = random.randint(0, len(_circle_geoms_)-1)                
cx, cy, r  = _circle_geoms_[c0]
a0         = random.random() * 2 * pi
_entry_pt_ = (cx+(r+_radius_inc_test_+0.5)*cos(a0),cy+(r+_radius_inc_test_+0.5)*sin(a0),c0)
                
# Randomize the exit points
_exit_pts_ = []
for i in range(_n_paths_):
    c1 = random.randint(0,len(_circle_geoms_)-1)
    while c1 == c0:
        c1 = random.randint(0,len(_circle_geoms_)-1)
    cx, cy, r  = _circle_geoms_[c1]
    a1         = random.random() * 2 * pi
    _exit_pts_.append((cx+(r+_radius_inc_test_+0.5)*cos(a1),cy+(r+_radius_inc_test_+0.5)*sin(a1),c1))

svg = '<svg x="0" y="0" width="600" height="400"><rect x="0" y="0" width="600" height="400" fill="#ffffff" />'
# Render Cirlces
for _geom_ in _circle_geoms_:
    svg += f'<circle cx="{_geom_[0]}" cy="{_geom_[1]}" r="{_geom_[2]}" stroke="#c0c0c0" fill="#404040" fill-opacity="0.2" />'
# Render Entry Points
svg += f'<circle cx="{_entry_pt_[0]}" cy="{_entry_pt_[1]}" r="3" stroke="#00af00" fill="#00af00" />'
for _exit_ in _exit_pts_:
    svg += f'<circle cx="{_exit_[0]}" cy="{_exit_[1]}" r="3" stroke="#ff0000" fill="#ff0000" />'
svg_base = svg
rt.displaySVG(svg_base + '</svg>')

In [ ]:
def circularPathRouter(entry_pt,                # (x,y,circle_i) -- where circle_i is the circle index from circle_geoms
                       exit_pts,                # [(x,y,circle_i),(x,y,circle_i),(x,y,circle_i), ...] -- where circle_i is the circle index from circle_geoms
                       circle_geoms,            # [(cx,cy,r),(cx,cy,r), ...]
                       escape_px        = 5,    # length to push the exit points (and entry point) away from circle
                       min_circle_sep   = 30,   # minimum distance between circles
                       half_sep         = 15,   # needs to be more than the radius_inc_test ... half separation (but doesn't have to be)
                       radius_inc_test  = 4,    # for routing around circles, how much to test with
                       radius_start     = 5):   # needs to be more than the radius_inc_test ... less than the min_circle_sep
    # Calculate a path around the circle geometries
    def calculatePathAroundCircles(pts):
        def breakSegment(_segment_):
            if rt.segmentLength(_segment_) < 2.0:
                return _segment_
            for _geom_ in circle_geoms:
                _circle_plus_ = (_geom_[0], _geom_[1], _geom_[2]+radius_inc_test)
                _dist_, _inter_  = rt.segmentIntersectsCircle(_segment_,_circle_plus_)
                if _dist_ <= _circle_plus_[2]:
                    if _inter_[0] == _geom_[0] and _inter_[1] == _geom_[1]:
                        dx, dy   = _segment_[1][0] - _segment_[0][0], _segment_[1][1] - _segment_[0][1]
                        l        = sqrt(dx*dx+dy*dy)
                        dx,  dy  = dx/l, dy/l
                        pdx, pdy = -dy, dx 
                        return [(_segment_[0][0], _segment_[0][1]), (_geom_[0] + pdx*(_geom_[2]+half_sep), _geom_[1] + pdy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
                    else:
                        dx, dy = _inter_[0] - _geom_[0], _inter_[1] - _geom_[1]
                        l      = sqrt(dx*dx+dy*dy)
                        dx, dy = dx/l, dy/l
                        return [(_segment_[0][0], _segment_[0][1]), (_geom_[0]  + dx*(_geom_[2]+half_sep), _geom_[1]  + dy*(_geom_[2]+half_sep)), (_segment_[1][0], _segment_[1][1])]
            return _segment_
        last_length = 0
        _segments_  = []
        for _pt_ in pts:
            _segments_.append(_pt_)
        while last_length != len(_segments_):
            last_length    = len(_segments_)
            _new_segments_ = []
            for i in range(len(_segments_)-1):
                _new_ = breakSegment([_segments_[i], _segments_[i+1]])
                if len(_new_) == 3:
                    _new_segments_.append(_new_[0])
                    _new_segments_.append(_new_[1])
                else:
                    _new_segments_.append(_new_[0])
            _new_segments_.append(_new_[-1])
            _segments_ = _new_segments_        
        return _segments_
    
    # Fix up the the entry and exit points...
    x_min,y_min,x_max,y_max = entry_pt[0],entry_pt[1],entry_pt[0],entry_pt[1]
    entries = []
    x0,y0,ci  = entry_pt
    uv        = rt.unitVector(((circle_geoms[ci][0],circle_geoms[ci][1]),(x0,y0)))
    x0s,y0s   = x0+uv[0]*escape_px, y0+uv[1]*escape_px
    for pt in exit_pts:
        x1,y1,ci  = pt        
        uv        = rt.unitVector(((circle_geoms[ci][0],circle_geoms[ci][1]),(x1,y1)))
        x1s,y1s   = x1+uv[0]*escape_px, y1+uv[1]*escape_px
        entries.append([(x0,y0), (x0s,y0s), (x1s,y1s), (x1,y1)])
        x_min,y_min,x_max,y_max = min(x_min,x1),min(y_min,y1),max(x_max,x1),max(y_max,y1)
        x_min,y_min,x_max,y_max = min(x_min,x1s),min(y_min,y1s),max(x_max,x1s),max(y_max,y1s)

    # XY Quad Tree
    xy_tree = rt.xyQuadTree((x_min-half_sep,y_min-half_sep,x_max+half_sep,y_max+half_sep))

    # Sort paths by length (longest first)
    exit_sorter = []
    for i in range(len(entries)):
        _entry_ = entries[i]
        l = rt.segmentLength((_entry_[0], _entry_[3]))
        exit_sorter.append((l,i))
    exit_sorter = sorted(exit_sorter)
    exit_sorter.reverse()

    # keep track of all of the final paths
    paths = []
    for i in range(len(entries)):
        paths.append(entries[i])

    # plot out the longest path
    i_longest        = exit_sorter[0][1]
    pts              = entries[i_longest]
    _path_           = calculatePathAroundCircles(pts)
    _path_smooth_    = rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))
    paths[i_longest] = _path_smooth_
    for i in range(len(_path_smooth_)):
        pt = (_path_smooth_[i][0], _path_smooth_[i][1], i_longest, i)
        xy_tree.add([pt])

    # analyze the other paths
    for i in range(1,len(exit_sorter)):
        i_path        =  exit_sorter[i][1]
        pts           =  entries[i_path]
        _path_        =  calculatePathAroundCircles(pts)
        _path_smooth_ =  rt.smoothSegments(rt.expandSegmentsIntoPiecewiseCurvedParts(_path_, amp=5.0, ampends=8.0, max_travel=1))    
        # merge with existing path
        _path_merged_ =  [_path_smooth_[-1]]
        for j in range(len(_path_smooth_)-2, 2, -1):
            closest = xy_tree.closest((_path_smooth_[j][0],_path_smooth_[j][1]), n=1)
            _path_merged_.append(_path_smooth_[j])
            if closest[0][0] < 5:
                _path_merged_.append((closest[0][1][0], closest[0][1][1]))
                break
        # save the path off
        paths[i_path] = _path_merged_
        # update xy tree
        for j in range(len(_path_merged_)-3):
            pt = (_path_merged_[j][0], _path_merged_[j][1], i_path, j)
            xy_tree.add([pt])
    # return the merged paths            
    return paths

def svgPathTag(_segments_, stroke='#000000', stroke_width=0.4):
    d = f'M {_segments_[0][0]} {_segments_[0][1]}'
    for i in range(1,len(_segments_)):
        d += f' L {_segments_[i][0]} {_segments_[i][1]}'
    return f'<path d="{d}" stroke="{stroke}" stroke-width="{stroke_width}" fill="none"/>'

_paths_     = circularPathRouter(_entry_pt_,_exit_pts_,_circle_geoms_)
_paths_svg_ = []
for _path_ in _paths_:
    _paths_svg_.append(svgPathTag(_path_))
rt.displaySVG(svg_base + ''.join(_paths_svg_) + '</svg>')